**Author**: 

Abdelrahman Khairy Mahmoud

&emsp; M.Eng. in Mechanical and Industrial Engineering

&emsp; University of Toronto, Toronto, ON, Canada

**Project description**:

The purpose of this project is to perform web-scraping on Bilgi University's (Istanbul, Turkey) curriculum for the Management Information Systems (MIS) program.

**Notes**:


- All curriculum webpages observed essentially follow the same outline format. An intended later version of this project will be adapted to obtain data pertaining to any course on the University website
- Later versions will support individual course content data mining 

In [1]:
# importing necessary libraries
import time
import requests as re
from bs4 import BeautifulSoup as bs

In [2]:
# browser header for requests object
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36 Edg/99.0.1150.46"}

# homepage for MIS curriculum
home = re.get('https://ects.bilgi.edu.tr/Department/Curriculum?catalog_departmentId=126632', headers)
time.sleep(3) # time delay to ensure all page contents are loaded

# Create a beautifulSoup object with the webpage's HTML
soup = bs(home.text, 'html.parser') 

In [3]:
container = soup.find("div", {'class':"panel panel-default"})
container2 = container.find("div", {'class':'panel-body'})

# div tags contain table headings
divs = container2.find_all("div")
divs = [div.text for div in divs]

# tables ontain the core curriculum content (incl. electives)
tables = container2.find_all("table")
tables = [table.find("tbody") for table in tables]

In [4]:
divs

['Level : 1 | Semester : 1 - 52. Management Information Systems',
 'Level : 1 | Semester : 2 - 52. Management Information Systems',
 'Level : 2 | Semester : 1 - 52. Management Information Systems',
 'Level : 2 | Semester : 2 - 52. Management Information Systems',
 'Level : 3 | Semester : 1 - 52. Management Information Systems',
 'Level : 3 | Semester : 2 - 52. Management Information Systems',
 'Level : 4 | Semester : 1 - 52. Management Information Systems',
 'Level : 4 | Semester : 2 - 52. Management Information Systems',
 '\nTotal ECTS : 240\n']

In [5]:
rows = [0]*len(tables)
rows = [table.find_all("tr") for table in tables]

In [11]:
tables_container = {}
for index, table in enumerate(tables):
    label = divs[index]
    tables_container[label] = table

In [7]:
rows_container = {}
for semester, table in tables_container.items():
    rows_container[semester] = table.find_all()

8

In [34]:
tables[4].find_all("tr")[0].find_all("td")[2].text.strip()

'Core'